##**End-to-End Test**
(จากข้อมูลใน drive ที่แนบมาลองว่ารันได้ไหม)

#ลงคำสั่งต่างๆ

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# ลง Library ที่จำเป็น (มีเครื่องหมายตกใจ ! นำหน้าด้วยนะ)
!pip install pydicom SimpleITK opencv-python-headless matplotlib

##แปลงข้อมูลจากไฟล์ DICOM (CT images, RTSTRUCT contours, RTDOSE grid) ให้เป็นไฟล์ Numpy Array (.npy)

Align ให้ตรงกับ Grid ของ CT เรียบร้อย

In [19]:
import os
import numpy as np
import pydicom
import cv2
import logging
import traceback
from pathlib import Path

# Setup Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class BrachyDataPreprocessor:
    def __init__(self, output_dir, target_rois):
        self.output_dir = Path(output_dir)
        self.target_rois = [roi.lower().strip() for roi in target_rois]
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def _find_dicom_files(self, folder_path, modality):
        """ค้นหาไฟล์ DICOM แบบ Recursive"""
        found_files = []
        folder_path = Path(folder_path)

        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith('.dcm'):
                    file_path = os.path.join(root, file)
                    try:
                        # อ่าน Header ก่อนเพื่อความเร็ว
                        dcm = pydicom.dcmread(file_path, stop_before_pixels=True)
                        if hasattr(dcm, 'Modality') and dcm.Modality == modality:
                            found_files.append(pydicom.dcmread(file_path))
                    except Exception:
                        continue

        if modality == 'CT':
            # เรียง CT ตามแกน Z
            if found_files:
                found_files.sort(key=lambda x: float(x.ImagePositionPatient[2]))
            return found_files
        elif len(found_files) > 0:
            return found_files[0]
        return None

    def process_ct(self, ct_slices):
        """แปลง CT slices เป็น 3D Numpy Array"""
        try:
            rows, cols = ct_slices[0].Rows, ct_slices[0].Columns
            depth = len(ct_slices)
            volume = np.zeros((depth, rows, cols), dtype=np.float32)

            for i, s in enumerate(ct_slices):
                slope = getattr(s, 'RescaleSlope', 1.0)
                intercept = getattr(s, 'RescaleIntercept', 0.0)
                volume[i] = s.pixel_array.astype(np.float32) * slope + intercept

            return volume
        except Exception as e:
            logger.error(f"❌ Error processing CT: {e}")
            return None

    def create_masks(self, rs_file, ct_slices, ct_shape):
        """สร้าง Mask จาก RTSTRUCT"""
        masks_dict = {}

        z_positions = np.array([float(s.ImagePositionPatient[2]) for s in ct_slices])
        origin = ct_slices[0].ImagePositionPatient
        spacing = ct_slices[0].PixelSpacing

        for roi in rs_file.StructureSetROISequence:
            name = roi.ROIName.lower().strip()
            if name not in self.target_rois:
                continue

            roi_number = roi.ROINumber
            mask = np.zeros(ct_shape, dtype=np.uint8)
            contour_sequence = None

            # หา Contour Sequence ที่ตรงกับ ROI Number
            for contour in rs_file.ROIContourSequence:
                if contour.ReferencedROINumber == roi_number:
                    if hasattr(contour, 'ContourSequence'):
                        contour_sequence = contour.ContourSequence
                    break

            if not contour_sequence:
                continue


            for contour_slice in contour_sequence:
                if not hasattr(contour_slice, 'ContourData'): continue

                points = np.array(contour_slice.ContourData).reshape(-1, 3)
                z_coord = points[0, 2]

                # หา Slice ที่ใกล้เคียงที่สุด (Nearest Neighbor)
                dist = np.abs(z_positions - z_coord)
                slice_idx = np.argmin(dist)

                if dist[slice_idx] > 2.5:
                    continue

                # แปลงพิกัดโลก (mm) -> พิกัดภาพ (pixel)
                x_pixel = np.round((points[:, 0] - origin[0]) / spacing[0]).astype(np.int32)
                y_pixel = np.round((points[:, 1] - origin[1]) / spacing[1]).astype(np.int32)

                pixel_coords = np.stack([x_pixel, y_pixel], axis=1)
                cv2.fillPoly(mask[slice_idx], [pixel_coords], 1)

            masks_dict[name] = mask
        return masks_dict

    def align_dose(self, dose_dcm, ct_slices):
        """
        ✅ บังคับให้ Dose วิ่งเข้าหา Grid ของ CT โดยคำนวณพิกัดเอง
        """
        logger.info("🔄 Aligning dose using MANUAL TRILINEAR INTERPOLATION...")

        try:
            # 1. เตรียมข้อมูล Dose
            dose_array = dose_dcm.pixel_array.astype(np.float32) * getattr(dose_dcm, 'DoseGridScaling', 1.0)

            # พิกัดของ Dose Grid
            dose_origin = np.array(dose_dcm.ImagePositionPatient, dtype=np.float32)
            dose_pixel_spacing = np.array(dose_dcm.PixelSpacing, dtype=np.float32)
            if hasattr(dose_dcm, 'GridFrameOffsetVector'):
                z_diffs = np.diff(dose_dcm.GridFrameOffsetVector)
                dose_z_spacing = z_diffs[0] if len(z_diffs) > 0 else 1.0
            else:
                dose_z_spacing = 1.0 # Default

            dose_spacing = np.array([dose_z_spacing, dose_pixel_spacing[1], dose_pixel_spacing[0]])
            dose_shape = dose_array.shape

            # 2. เตรียมข้อมูล CT (Target Grid)
            ct_first = ct_slices[0]
            ct_origin = np.array(ct_first.ImagePositionPatient, dtype=np.float32)
            ct_pixel_spacing = np.array(ct_first.PixelSpacing, dtype=np.float32)

            # คำนวณ Z spacing ของ CT
            if len(ct_slices) > 1:
                ct_z_spacing = abs(float(ct_slices[1].ImagePositionPatient[2]) - float(ct_slices[0].ImagePositionPatient[2]))
            else:
                ct_z_spacing = 1.0

            ct_spacing = np.array([ct_z_spacing, ct_pixel_spacing[1], ct_pixel_spacing[0]])
            ct_shape = (len(ct_slices), ct_first.Rows, ct_first.Columns)

            # 3. สร้าง Grid เปล่าๆ ขนาดเท่า CT
            aligned_dose = np.zeros(ct_shape, dtype=np.float32)

            # 4. สร้างตารางพิกัด (Meshgrid) ของ CT
            # นี่คือหัวใจสำคัญ: "สำหรับทุกๆ pixel ใน CT... มันตรงกับจุดไหนใน Dose?"
            z_idx, y_idx, x_idx = np.meshgrid(
                np.arange(ct_shape[0]),
                np.arange(ct_shape[1]),
                np.arange(ct_shape[2]),
                indexing='ij'
            )

            # 5. แปลงพิกัด Index CT -> Physical World (mm)
            world_z = ct_origin[2] + z_idx * ct_spacing[0]
            world_y = ct_origin[1] + y_idx * ct_spacing[1]
            world_x = ct_origin[0] + x_idx * ct_spacing[2]

            # 6. แปลงพิกัด Physical World -> Index Dose (ทศนิยม)
            dose_z_idx = (world_z - dose_origin[2]) / dose_spacing[0]
            dose_y_idx = (world_y - dose_origin[1]) / dose_spacing[1]
            dose_x_idx = (world_x - dose_origin[0]) / dose_spacing[2]

            # 7. กรองเฉพาะจุดที่ตกอยู่ในกล่อง Dose (จุดที่อยู่นอกกล่อง Dose ให้เป็น 0)
            valid_mask = (
                (dose_z_idx >= 0) & (dose_z_idx < dose_shape[0] - 1) &
                (dose_y_idx >= 0) & (dose_y_idx < dose_shape[1] - 1) &
                (dose_x_idx >= 0) & (dose_x_idx < dose_shape[2] - 1)
            )

            # 8. คำนวณ Trilinear Interpolation (เฉพาะจุด Valid)
            dz = dose_z_idx[valid_mask]
            dy = dose_y_idx[valid_mask]
            dx = dose_x_idx[valid_mask]

            z0 = np.floor(dz).astype(int)
            y0 = np.floor(dy).astype(int)
            x0 = np.floor(dx).astype(int)

            z1 = z0 + 1
            y1 = y0 + 1
            x1 = x0 + 1

            # น้ำหนัก (Weight) สำหรับ Interpolation
            w_z = dz - z0
            w_y = dy - y0
            w_x = dx - x0

            # ดึงค่าจาก 8 จุดรอบข้าง (Corner values)
            c000 = dose_array[z0, y0, x0]
            c001 = dose_array[z0, y0, x1]
            c010 = dose_array[z0, y1, x0]
            c011 = dose_array[z0, y1, x1]
            c100 = dose_array[z1, y0, x0]
            c101 = dose_array[z1, y0, x1]
            c110 = dose_array[z1, y1, x0]
            c111 = dose_array[z1, y1, x1]

            # สูตร Trilinear Interpolation
            interpolated_values = (
                c000 * (1 - w_z) * (1 - w_y) * (1 - w_x) +
                c001 * (1 - w_z) * (1 - w_y) * w_x +
                c010 * (1 - w_z) * w_y * (1 - w_x) +
                c011 * (1 - w_z) * w_y * w_x +
                c100 * w_z * (1 - w_y) * (1 - w_x) +
                c101 * w_z * (1 - w_y) * w_x +
                c110 * w_z * w_y * (1 - w_x) +
                c111 * w_z * w_y * w_x
            )

            # ยัดค่าที่คำนวณได้กลับลงไปใน Grid ของ CT
            aligned_dose[valid_mask] = interpolated_values

            logger.info("✅ Manual alignment completed successfully.")
            return aligned_dose

        except Exception as e:
            logger.error(f"❌ Error aligning dose: {e}")
            traceback.print_exc()
            return None

    def run_pipeline(self, patient_folder):
        patient_id = Path(patient_folder).name
        save_path = self.output_dir / patient_id
        save_path.mkdir(exist_ok=True)

        logger.info(f"🚀 Processing Patient: {patient_id}")

        ct_slices = self._find_dicom_files(patient_folder, 'CT')
        if not ct_slices:
            logger.error("No CT Found")
            return

        ct_vol = self.process_ct(ct_slices)
        np.save(save_path / "CT_raw.npy", ct_vol)

        rs_file = self._find_dicom_files(patient_folder, 'RTSTRUCT')
        if rs_file:
            masks = self.create_masks(rs_file, ct_slices, ct_vol.shape)
            combined_mask = np.zeros_like(ct_vol, dtype=np.uint8)
            for name, mask_data in masks.items():
                np.save(save_path / f"Mask_{name}.npy", mask_data)
                combined_mask = np.logical_or(combined_mask, mask_data).astype(np.uint8)
            np.save(save_path / "Combined_Mask.npy", combined_mask)

        dose_file = self._find_dicom_files(patient_folder, 'RTDOSE')
        if dose_file:
            aligned_dose = self.align_dose(dose_file, ct_slices)
            if aligned_dose is not None:
                np.save(save_path / "Dose_raw.npy", aligned_dose)
                logger.info("💾 Saved Dose_raw.npy")

In [20]:
# ================= (แก้ตรงนี้!) =================
# 1. วาง Path
DICOM_DIR = "/content/drive/MyDrive/Medical Physics_PSCM05 (M. Sc.)/code/model/FSD1_1"  # <--- แก้บรรทัดนี้

# 2. ตั้งชื่อโฟลเดอร์ปลายทางที่จะเซฟไฟล์ .npy
OUTPUT_DIR = "/content/drive/MyDrive/Medical Physics_PSCM05 (M. Sc.)/code/model/Data_numpy/v2/Predict"

# 3. รายชื่ออวัยวะ/Applicator ที่ต้องการ
TARGETS = [
    'sigmoid_ct', 'bladder_ct', 'rectum_ct', 'bowel_ct', 'hrctv_ct',
    'Applicator1', 'Applicator2', 'Applicator3',
    'needdle1', 'needdle2'
]
# ========================================================
if __name__ == "__main__":
    print(f"📂 Reading from: {DICOM_DIR}")
    print(f"💾 Saving to: {OUTPUT_DIR}")

    processor = BrachyDataPreprocessor(OUTPUT_DIR, TARGETS)
    root_path = Path(DICOM_DIR)

    if not root_path.exists():
        print("❌ Error: ไม่เจอโฟลเดอร์ DICOM!")
    else:
        # เช็คว่าข้างในมีโฟลเดอร์ย่อยไหม (กรณีมีหลายคนไข้)
        subdirs = [d for d in root_path.iterdir() if d.is_dir()]

        if len(subdirs) > 0:
            print(f"👥 Found {len(subdirs)} patient folders.")
            for patient_dir in subdirs:
                processor.run_pipeline(str(patient_dir))
        else:
            print("👤 Processing single patient folder...")
            processor.run_pipeline(str(root_path))

    print("✅ เสร็จแล้ว! ")

📂 Reading from: /content/drive/MyDrive/Medical Physics_PSCM05 (M. Sc.)/code/model/FSD1_1
💾 Saving to: /content/drive/MyDrive/Medical Physics_PSCM05 (M. Sc.)/code/model/Data_numpy/v2/Predict
👤 Processing single patient folder...
✅ เสร็จแล้ว! 


#Preprocessing for Training

Clip HU [-1000, 2000], Rescale [0, 1], Normalize Dose [0, 1]

In [21]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
from pathlib import Path

class BrachyTrainingPrep:
    def __init__(self, input_dir, output_dir, min_hu=-1000, max_hu=2000):
        self.input_dir = Path(input_dir)
        self.output_dir = Path(output_dir)
        self.min_hu = min_hu
        self.max_hu = max_hu
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def load_npy(self, file_path):
        """Safe load numpy file"""
        if file_path.exists():
            return np.load(file_path)
        return None

    def clip_and_rescale_ct(self, ct_array):
        """Clip HU and Rescale to [0, 1]"""
        # 1. Clip
        clipped = np.clip(ct_array, self.min_hu, self.max_hu)
        # 2. Rescale Min-Max to [0, 1]
        rescaled = (clipped - self.min_hu) / (self.max_hu - self.min_hu)
        return rescaled.astype(np.float32)

    def normalize_dose(self, dose_array):
        """Normalize Dose to [0, 1]"""
        if dose_array.max() > 0:
            return (dose_array / dose_array.max()).astype(np.float32)
        return dose_array.astype(np.float32)

    def augment_data(self, ct, dose, mask):
        """Random Rotation & Flip"""
        # Random Rotate (-15 to 15 deg)
        if np.random.rand() > 0.5:
            angle = np.random.uniform(-15, 15)
            # Use order=1 (linear) for CT/Dose, order=0 (nearest) for Mask
            ct = rotate(ct, angle, axes=(1, 2), reshape=False, order=1)
            dose = rotate(dose, angle, axes=(1, 2), reshape=False, order=1)
            mask = rotate(mask, angle, axes=(1, 2), reshape=False, order=0)

        # Random Flip
        if np.random.rand() > 0.5:
            axis = np.random.choice([0, 1, 2])
            ct = np.flip(ct, axis=axis)
            dose = np.flip(dose, axis=axis)
            mask = np.flip(mask, axis=axis)

        return ct, dose, mask

    def visualize(self, ct_orig, dose_orig, ct_new, dose_new, save_path):
        """Compare Before vs After"""
        mid_slice = ct_orig.shape[0] // 2

        fig, ax = plt.subplots(2, 2, figsize=(10, 8))

        # Original
        ax[0,0].imshow(ct_orig[mid_slice], cmap='gray')
        ax[0,0].set_title(f"Original CT (HU {int(ct_orig.min())} to {int(ct_orig.max())})")
        ax[0,1].imshow(dose_orig[mid_slice], cmap='jet')
        ax[0,1].set_title(f"Original Dose (Max {dose_orig.max():.2f} Gy)")

        # Preprocessed
        ax[1,0].imshow(ct_new[mid_slice], cmap='gray')
        ax[1,0].set_title(f"Norm CT (0.0 - 1.0)")
        ax[1,1].imshow(dose_new[mid_slice], cmap='jet')
        ax[1,1].set_title(f"Norm Dose (0.0 - 1.0)")

        for a in ax.flatten(): a.axis('off')
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()

    def process_all(self, augment=False):
        """Main processing loop"""
        patients = [d for d in self.input_dir.iterdir() if d.is_dir()]
        print(f"🔄 Found {len(patients)} patients to process.")

        for patient_dir in patients:
            pid = patient_dir.name
            print(f"➡️ Processing: {pid}")

            # 1. Load Data
            ct = self.load_npy(patient_dir / "CT_raw.npy")
            dose = self.load_npy(patient_dir / "Dose_raw.npy")
            mask = self.load_npy(patient_dir / "Combined_Mask.npy")

            if ct is None or dose is None:
                print(f"⚠️ Missing CT or Dose for {pid}, skipping.")
                continue

            if mask is None: # Create empty mask if missing
                mask = np.zeros_like(ct)

            # 2. Preprocessing
            ct_norm = self.clip_and_rescale_ct(ct)
            dose_norm = self.normalize_dose(dose)
            mask_bin = (mask > 0.5).astype(np.float32)

            # 3. Augmentation (Optional)
            if augment:
                ct_norm, dose_norm, mask_bin = self.augment_data(ct_norm, dose_norm, mask_bin)

            # 4. Save
            save_p = self.output_dir / pid
            save_p.mkdir(parents=True, exist_ok=True)

            np.save(save_p / "CT_norm.npy", ct_norm)
            np.save(save_p / "Dose_norm.npy", dose_norm)
            np.save(save_p / "Mask_norm.npy", mask_bin)

            # 5. Visualize Check
            self.visualize(ct, dose, ct_norm, dose_norm, save_p / "check_preprocessing.png")

        print("✅ All Training Data Prepared!")

# ================= RUN =================
if __name__ == "__main__":
    # Path จากขั้นตอนที่แล้ว
    INPUT_DIR = "/content/drive/MyDrive/Medical Physics_PSCM05 (M. Sc.)/code/model/Data_numpy/v2/Predict"

    # Path ใหม่ที่จะเก็บข้อมูลพร้อมเทรน (Normalized Data)
    OUTPUT_TRAIN_DIR = "/content/drive/MyDrive/Medical Physics_PSCM05 (M. Sc.)/code/model/Data_numpy/v2/Data_Train_Ready"

    # เริ่มทำงาน
    prep = BrachyTrainingPrep(INPUT_DIR, OUTPUT_TRAIN_DIR)
    prep.process_all(augment=False) # เปลี่ยนเป็น True ถ้าอยากหมุนภาพเพิ่มข้อมูล

🔄 Found 1 patients to process.
➡️ Processing: FSD1_1
✅ All Training Data Prepared!
